In [1]:
import numpy as np
from PIL import Image
from io import BytesIO
import json
import base64
from encrusted import decode, encode 
import encrusted

In [2]:
def img_bytes_to_b64_str(img_bytes):
    img_bytes_b64 = base64.b64encode(img_bytes)
    img_str_b64 = img_bytes_b64.decode("ascii")
    return img_str_b64

def b64_str_to_img_bytes(b64_str):
    img_bytes_b64 = b64_str.encode("ascii")
    img_bytes = base64.b64decode(img_bytes_b64)
    return img_bytes

In [3]:
TEST_SIZE = (3, 512, 512)
TEST_TYPE = np.int32

In [4]:
# make quantized mask
arr = np.random.uniform(0, 1, TEST_SIZE)
quantized_arr = (arr * 256 // 1).astype(TEST_TYPE)

### numpy save and load

In [5]:
%%timeit
byte_strings = []
for mask in quantized_arr:
    np_bytes = BytesIO()
    np.save(np_bytes, mask, allow_pickle=True)
    byte_strings.append(
        {
            "mask":img_bytes_to_b64_str(np_bytes.getvalue())
        }
    )

4.86 ms ± 22.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
byte_strings = []
for mask in quantized_arr:
    np_bytes = BytesIO()
    np.save(np_bytes, mask, allow_pickle=True)
    byte_strings.append(
        {
            "mask":img_bytes_to_b64_str(np_bytes.getvalue())
        }
    )

In [7]:
%%timeit
masks = []
for byte_str in byte_strings:
    np_bytes = BytesIO()
    masks.append({
        "mask": np.load(BytesIO(b64_str_to_img_bytes(byte_str["mask"])), allow_pickle=True)
    })

8.45 ms ± 26.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
# assert lossless
for mask in quantized_arr:
    np_bytes = BytesIO()
    np.save(np_bytes, mask)
    byte_str = img_bytes_to_b64_str(np_bytes.getvalue())
    decoded_mask = np.load(BytesIO(b64_str_to_img_bytes(byte_str)))
    assert (decoded_mask == mask).all()

### encrusted

In [9]:
%%timeit
byte_strings = []
for mask in quantized_arr:
    byte_strings.append({
        "mask": encode(mask)
    })

16.2 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
byte_strings = []
for mask in quantized_arr:
    byte_strings.append({
        "mask": encode(mask)
    })

In [11]:
%%timeit
masks = []
for byte_str in byte_strings:
    masks.append({
        "mask": decode(byte_str["mask"], TEST_TYPE)
    })

11 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
# assert lossless
for mask in quantized_arr:
    img_str = encode(mask)
    decoded_mask = decode(img_str, TEST_TYPE)
    assert (decoded_mask == mask).all()

### png encoding

In [17]:
%%timeit
pngs = []
for mask in quantized_arr:
    img = Image.fromarray(mask)
    bio = BytesIO()
    img.save(bio, "PNG")
    pngs.append(
        {
            "mask":img_bytes_to_b64_str(bio.getvalue())
        }
    )

177 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
pngs = []
for mask in quantized_arr:
    img = Image.fromarray(mask)
    bio = BytesIO()
    img.save(bio, "PNG")
    pngs.append(
        {
            "mask":img_bytes_to_b64_str(bio.getvalue())
        }
    )

In [14]:
byte_strings = []
for mask in quantized_arr:
    byte_strings.append({
        "mask": encode(mask)
    })

In [15]:
[len(a["mask"])//1_000 for a in pngs]

[436, 436, 436]

In [16]:
[len(a["mask"])//1_000 for a in byte_strings]

[544, 544, 544]